In [1]:
%%time
import torch
import numpy as np
from scipy.optimize import minimize, Bounds
import cProfile, io, pstats
from pstats import SortKey

torch.manual_seed(0)
np.random.seed(0)

torch.set_default_dtype(torch.double)

bounds = Bounds(
    lb=np.zeros(128),
    ub=np.ones(128),
    keep_feasible=True,
)
x0 = np.random.random(128)
None

[KeOps] Warning : omp.h header is not in the path, disabling OpenMP. To fix this, you can set the environment
                  variable OMP_PATH to the location of the header before importing keopscore or pykeops,
                  e.g. using os.environ: import os; os.environ['OMP_PATH'] = '/path/to/omp/header'
[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode
CPU times: user 2.09 s, sys: 898 ms, total: 2.99 s
Wall time: 12.7 s


In [2]:
%%time

print("\n\n\n\n\n\n Optimize a simple root finding problem \n\n")

rand_tensor = torch.rand(1024, 128, 128)

def f_and_grad(x):
    X_tensor = torch.from_numpy(x).contiguous().requires_grad_(True)
    diff = rand_tensor - X_tensor
    loss = (diff @ diff).sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
        options={"maxfun": 20},
    )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)
None







 Optimize a simple root finding problem 


         3837 function calls (3832 primitive calls) in 2.718 seconds

   Ordered by: cumulative time
   List reduced from 262 to 26 due to restriction <0.1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.718    2.718 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_minimize.py:53(minimize)
        1    0.000    0.000    2.718    2.718 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:290(_minimize_lbfgsb)
       42    0.030    0.001    2.715    0.065 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_optimize.py:71(_compute_if_needed)
       22    0.000    0.000    2.715    0.123 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:293(_update_fun)
       21    0.000    0.000    2.715    0.129 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/sc

In [3]:
%%time

print("\n\n\n\n\n\n 20 reference evaluations of f_and_grad \n\n")

X = np.random.random(128)

with cProfile.Profile() as pr:
    for _ in range(20):
        f_and_grad(X)







 20 reference evaluations of f_and_grad 


CPU times: user 11.6 s, sys: 4.63 s, total: 16.2 s
Wall time: 2.48 s
